In [9]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_7575/945712042.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import pandas as pd
import os
import argparse
from typing import Any, Callable, Sequence
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.flatten_util import ravel_pytree
import numpy as np
import matplotlib.pyplot as plt
from flax import linen as nn


from bong.util import run_rebayes_algorithm, gaussian_kl_div, MLP
from bong.src import bbb, blr, bog, bong, experiment_utils
#from bong.agents import AGENT_NAMES




Matplotlib created a temporary cache directory at /tmp/matplotlib-8w3txzq1 because the default path (/teamspace/studios/this_studio/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [24]:
parser = argparse.ArgumentParser()

# Data parameters
parser.add_argument("--dataset", type=str, default="sarcos") 
parser.add_argument("--data_dim", type=int, default=0)
parser.add_argument("--dgp_type", type=str, default="na") # or mlp
parser.add_argument("--dgp_str", type=str, default="na") # 20_20_1 
parser.add_argument("--emission_noise", type=float, default=-1)
parser.add_argument("--ntrain", type=int, default=100)
parser.add_argument("--nval", type=int, default=100)
parser.add_argument("--ntest", type=int, default=100)
parser.add_argument("--linreg_baseline", type=int, default=0)


# Model parameters
#parser.add_argument("--agent", type=str, default="bong_fc", choices=AGENT_NAMES)
parser.add_argument("--algo", type=str, default="bong")
parser.add_argument("--param", type=str, default="fc")
parser.add_argument("--lr", type=float, default=0.01)
parser.add_argument("--niter", type=int, default=10) 
parser.add_argument("--nsample", type=int, default=100) 
parser.add_argument("--ef", type=int, default=1)
parser.add_argument("--lin", type=int, default=0)
parser.add_argument("--rank", type=int, default=10)
parser.add_argument("--model_type", type=str, default="lin") # or mlp
parser.add_argument("--model_str", type=str, default="")
parser.add_argument("--use_bias", type=int, default=1) 
parser.add_argument("--use_bias_layer1", type=int, default=1) 
parser.add_argument("--init_var", type=float, default=0.1)

# results
parser.add_argument("--dir", type=str, default="", help="directory to store results") 
parser.add_argument("--debug", type=bool, default=False)
parser.add_argument("--key", type=int, default=0)

parser.add_argument("--add_ones", type=int, default=1)

args = parser.parse_args([])
print(args)

Namespace(dataset='sarcos', data_dim=0, dgp_type='na', dgp_str='na', emission_noise=-1, ntrain=100, nval=100, ntest=100, linreg_baseline=0, algo='bong', param='fc', lr=0.01, niter=10, nsample=100, ef=1, lin=0, rank=10, model_type='lin', model_str='', use_bias=1, use_bias_layer1=1, init_var=0.1, dir='', debug=False, key=0, add_ones=1)


In [25]:
from do_job import *
from bong.util import run_rebayes_algorithm, get_gpu_name
#from bong.agents import AGENT_DICT, AGENT_NAMES, parse_agent_full_name, make_agent_name_from_parts
from bong.agents import make_agent_constructor
from datasets import make_dataset
from models import make_model

if isinstance(args.key, int):
    key = jr.PRNGKey(args.key)
else:
    key = args.key
key, subkey = jr.split(key)
key, data = make_dataset(subkey, args)
key, subkey = jr.split(key)
key, model = make_model(subkey, args, data)


constructor = make_agent_constructor(args.algo, args.param)
key, subkey = jr.split(key)
agent = constructor(
                    **model['model_kwargs'],
                    agent_key = subkey,
                    learning_rate = args.lr,
                    num_iter = args.niter,
                    num_samples = args.nsample,
                    linplugin = args.lin,
                    empirical_fisher = args.ef,
                    rank = args.rank
                )
print(agent)

key, subkey = jr.split(key)
results, elapsed, summary = run_agent(subkey, agent, data, model)
df = pd.DataFrame(results)

get sarcos 100 100 100
RebayesAlgorithm(init=<function fg_bong.__new__.<locals>.init_fn at 0x7fc7ce99a560>, predict=<function fg_bong.__new__.<locals>.pred_fn at 0x7fc7ce999fc0>, update=<function fg_bong.__new__.<locals>.update_fn at 0x7fc7cdc0b490>, sample=<function sample_fg_bong at 0x7fc857d56d40>, name='bong-fc-EF100', full_name='bong-fc-EF100')
Running bong-fc-EF100 + lin_1[P=22] on sarcos
Using GPU of type:  None
Time 1.27s
KL: 11412.0908
Test NLL: 9.6612,  NLPD: 9.6609
Val NLL 9.5079,  NLPD: 9.5216


In [3]:
expt = 'iter-bbb2'
srcdir = f'~/jobs/paper-{expt}/figs'
dstdir = f'~/bong/bong/results'
for suffix in ['pdf', 'png']:
    #src = f'{srcdir}/nlpd_smoothed_jitter_best_lr.{suffix}'
    src = f'{srcdir}/nlpd_smoothed_jitter_best_lr_250.{suffix}'
    #src = f'{srcdir}/nlpd_smoothed_jitter.{suffix}'
    dst = f'{dstdir}/{expt}-nlpd.{suffix}'
    cmd = f'cp {src} {dst}'
    print(cmd)
    os.system(cmd)

cp ~/jobs/paper-iter-bbb2/figs/nlpd_smoothed_jitter_best_lr_250.pdf ~/bong/bong/results/iter-bbb2-nlpd.pdf
cp ~/jobs/paper-iter-bbb2/figs/nlpd_smoothed_jitter_best_lr_250.png ~/bong/bong/results/iter-bbb2-nlpd.png


In [34]:
results_dir = '/teamspace/studios/this_studio/jobs/paper-algo-dlr-medium-n10k-lin-rank'

df = pd.read_csv(f'{results_dir}/jobs.csv')
df.head(n=20)

,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,model_str,key,dataset,data_dim,dgp_type,dgp_str,model_type,ntrain,ntest
0,paper-algo-dlr-medium-n10k-lin-rank-00,bong,diag,1,99,99,99,99,99.000,30_30_1,0,sarcos,0,na,na,mlp,10000,500
1,paper-algo-dlr-medium-n10k-lin-rank-01,bong,dlr,1,1,99,99,99,99.000,30_30_1,0,sarcos,0,na,na,mlp,10000,500
2,paper-algo-dlr-medium-n10k-lin-rank-02,bong,dlr,1,10,99,99,99,99.000,30_30_1,0,sarcos,0,na,na,mlp,10000,500
3,paper-algo-dlr-medium-n10k-lin-rank-03,bog,diag,1,99,99,99,99,0.005,30_30_1,0,sarcos,0,na,na,mlp,10000,500
4,paper-algo-dlr-medium-n10k-lin-rank-04,bog,diag,1,99,99,99,99,0.010,30_30_1,0,sarcos,0,na,na,mlp,10000,500
5,paper-algo-dlr-medium-n10k-lin-rank-05,bog,diag,1,99,99,99,99,0.050,30_30_1,0,sarcos,0,na,na,mlp,10000,500
6,paper-algo-dlr-medium-n10k-lin-rank-06,bog,diag,1,99,99,99,99,0.100,30_30_1,0,sarcos,0,na,na,mlp,10000,500
7,paper-algo-dlr-medium-n10k-lin-rank-07,bog,diag,1,99,99,99,99,0.500,30_30_1,0,sarcos,0,na,na,mlp,10000,500
8,paper-algo-dlr-medium-n10k-lin-rank-08,bog,dlr,1,1,99,99,99,0.005,30_30_1,0,sarcos,0,na,na,mlp,10000,500
9,paper-algo-dlr-medium-n10k-lin-rank-09,bog,dlr,1,10,99,99,99,0.005,30_30_1,0,sarcos,0,na,na,mlp,10000,500


In [ ]:
#condition = (df['minscore'] != df[args.metric])
condition = (df['lin']==1) | (df['ef']==0)
indices_to_drop = df[condition].index
df_filtered = df.drop(indices_to_drop)

df_filtered.head(n=20)

In [1]:
D = 22 # since add one for sarcos
# For FC
mlp = [(D+1)*n + (n+1)*1 for n in [10, 20, 30, 40]] # 1 layer
print(mlp)

# For DLR
mlp = [(D+1)*n + (n+1)*n  + (n+1)*1 for n in [10, 20, 30]] # 2 layer MLP
print(mlp)

mlp = [(D+1)*n + (n+1)*n + (n+1)*n + (n+1)*1 for n in [10, 20, 30]] # 3 layer MLP
print(mlp)


mlp=  (784+1)*50 + (50+1)*50  + (50+1)*10
print(mlp)



[241, 481, 721, 961]
[351, 901, 1651]
[461, 1321, 2581]
42310
